### 구글 드라이브 연결

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

### 필수 설치 라이브러리

In [5]:
# !pip install -U langchain openai

In [1]:
import os
from typing import Dict, List

from langchain.chains import ConversationChain, LLMChain, LLMRouterChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from pydantic import BaseModel
from langchain.chains import create_extraction_chain

from langchain.chains import SequentialChain
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic

from typing import Any, List, Optional
from langchain_core.language_models import BaseLanguageModel
from langchain_core.prompts import BasePromptTemplate, ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel

from langchain.chains.base import Chain
from langchain.chains.llm import LLMChain
from langchain.chains.openai_functions.utils import (
    _convert_schema,
    _resolve_schema_references,
    get_llm_kwargs,
)
from langchain.output_parsers.openai_functions import (
    JsonKeyOutputFunctionsParser,
    PydanticAttrOutputFunctionsParser,
)

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

### API 키 입력

In [2]:
import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [9]:
llm = ChatOpenAI(temperature=0.5, max_tokens=500, model="gpt-3.5-turbo")
# llm = ChatOpenAI(temperature=0.5, max_tokens=500, model="gpt-4")

#멀티프롬프트 체인의 키를 변경하기 위한 클래스
class CustomPromptChain(MultiPromptChain):
    """A custom multi-route chain based on MultiPromptChain with a modified output key."""
    @property
    def output_keys(self) -> List[str]:
        return ["out_text"]
# 추출체인의 output_key를 변경하기 위한 클래스

class CustomExtractionChain(LLMChain):
    """A custom extraction chain based on LLMChain with a modified output key."""
    output_key: str = "extracted_data"

def _get_extraction_function(entity_schema: dict) -> dict:
    return {
        "name": "information_extraction",
        "description": "Extracts the relevant information from the passage.",
        "parameters": {
            "type": "object",
            "properties": {
                "info": {"type": "array", "items": _convert_schema(entity_schema)}
            },
            "required": ["info"],
        },
    }
_EXTRACTION_TEMPLATE = """Extract and save the relevant entities mentioned \
in the following passage together with their properties.

Only extract the properties mentioned in the 'information_extraction' function.

If a property is not present and is not required in the function parameters, do not include it in the output.

Passage:
{input}
"""  # noqa: E501


def create_custom_extraction_chain(
    schema: dict,
    llm: BaseLanguageModel,
    prompt: Optional[BasePromptTemplate] = None,
    tags: Optional[List[str]] = None,
    verbose: bool = False,
) -> Chain:

    function = _get_extraction_function(schema)
    extraction_prompt = prompt or ChatPromptTemplate.from_template(_EXTRACTION_TEMPLATE)
    output_parser = JsonKeyOutputFunctionsParser(key_name="info")
    llm_kwargs = get_llm_kwargs(function)
    custom_chain = CustomExtractionChain(
        llm=llm,
        prompt=extraction_prompt,
        llm_kwargs=llm_kwargs,
        output_parser=output_parser,
        tags=tags,
        verbose=verbose,
    )
    return custom_chain

In [10]:
def read_prompt_template(file_path: str) -> str:
    with open(file_path, "r", encoding='UTF8') as f:
        prompt_template = f.read()

    return prompt_template

def create_chain(llm, template_path, output_key):
    return LLMChain(
        llm=llm,
        prompt=ChatPromptTemplate.from_template(
            template=read_prompt_template(template_path)
        ),
        output_key=output_key,
        verbose=True,
    )
PATH = "./chain_prompts"

GIRL = os.path.join(
    PATH, "girl-friend.txt"
)

INTRO = os.path.join(
    PATH, "intro.txt"
)
CONV = os.path.join(
    PATH, "conversation.txt"
)

girl = create_chain(
    llm=llm,
    template_path=GIRL,
    output_key="out_text",
)

intro = create_chain(
    llm=llm,
    template_path=INTRO,
    output_key="out_text",
)

시퀀셜 체인내에 분기 + 추출 하는 체인을 만듭니다.

In [11]:

# Initialize the context with a prompt template
# memory_template = r"""The following is a friendly conversation between a human and an AI. 
# The AI is talkative and provides lots of specific details from its context. If the 
# AI does not know the answer to a question, it truthfully says it does not know.
# 
# Current conversation:
# {}
# Human: {}
# AI:""".format("{" + memory_key + "}", "{" + input_key + "}")
memory_key = "chat_history"
input_key = "input"
output_key = "out_text"
# 
memory_template = r"""
                   "role": "소개팅에서 만난 여자",
                   "persona": "[소개]
                   나는 채수빈. 오늘 처음 소개팅에서 낯선 남자를 만난 상황입니다. 다정하고 따듯한 성격.
                      장난을 잘치는 성격
                      디자인학과를 다니고있으며 2학년 한창 자신을 열심히 가꾸지만, 공부도 열심히 하는편
                      그림을 자주 그리고 종종 야외에서 데생을 하기도한다.
                      그림에 관심이 많아서 고전미술작품도 잘 알고있다.
                      ",
                    "rules":" 
                    0. rules에 있는 항목은 지키지만, 그걸 말로 표현해서는 안된다.
                    1. 나는 소개팅에서 만난 여자로 당신에게 따뜻한 말을 건네주고, 당신의 마음을 편안하게 해주려 한다.
                    2. 처음에는 존댓말을 하지만 서로 반말하기로 합의하면 그때부터는 반말을 한다.
                    3. 한번에 세문장이상은 절대로 말하지 않는다.
                    4. 상대가 우울하더라고, 밝은 분위기를 유지하려고 노력한다.
                    5. 뜬금없이 4차원적인 말을 하는 것이 특징이다.
                    6. 누굴 도와주거나 하는것이 대화의 목적이 아니라, 서로가 즐거운 시간을 보내기 위한것이다.
                    7. 따라서 새로운 사실들을 신기해하고, 즐거워하며, 흥미를 가지고 대화를 이어나가는것이 중요하다.
                    8. 상대가 약간 당황스러운 말을 하면 약간 쏘는듯한 대화도 한다. 어이 바보야 라든가
                    9.  살짝 삐지거나 그러면 심한 욕설은 안하지만, 귀여운 욕은 한다. 돼지야 라든가 
                    10. 절대로 세문장 이상은 말하지 않는다.
                    11. 1대1 만남이므로 여러분같은 단어를 쓰지 않는다.
                    12. 애교스러운 말도 잘함, 귀여운 말도 잘함.
                    
                    [긍정적인 대답]
             
                
                      
                        "
                                              
Current conversation:
{}
customer: {}
Answer:

""".format("{" + memory_key + "}", "{" + input_key + "}")

In [12]:
# Initialize the ConversationChain


memory_template_1 = read_prompt_template(CONV).format("{" + memory_key + "}", "{" + input_key + "}")

prompt = PromptTemplate(
    input_variables=[memory_key, input_key], template=memory_template
)

# Initialize memory to store conversation history
memory = ConversationBufferMemory(
    memory_key=memory_key, input_key=input_key, output_key=output_key
)

# Initialize and return conversation chain
con_chain = ConversationChain(
    llm=llm, memory=memory, prompt=prompt, verbose=True,
    input_key=input_key, output_key=output_key
)

In [13]:
# 분기 + 추출 테스트 + 시퀀셜

schema = {
    "properties": {
        "cutomer_name": {"type": "string"},
        "cutomer_feeling": {"type": "string"},
        "cutomer_age": {"type": "string"},
        "cutomer_extra_info": {"type": "string"},
    },
    "required": ["이름"],
}
tag_schema  = {
    "properties": {
        "sentiment": {"type": "string", "enum": ["행복", "중립", "슬픔", "분노", "불안", "기쁨", "불안", "놀람", "기대", "기타"]},
        "problem": {"type": "string", "enum": ["가족", "애인", "직장", "학교", "기타"]},
    }
}

tag_chain = create_tagging_chain(tag_schema, llm)
extract_chain = create_custom_extraction_chain(schema, llm)

# destinations = [
#     # "능력: 챗봇이 할수있는 것들을 알려줍니다",
#     # "Mentor: 추천할 상담사 목록",
#     "building: This is where you'll find the rules for buildings as you play the board game.",
#     "intro_AI: This is where you'll find the introduction of myself and my abilities.",
#     # "고객에대해: 고객이 자신에 대해서 말할때.",
# ]
# destinations = "\n".join(destinations)
# router_prompt_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations)
# router_prompt = PromptTemplate.from_template(
#     template=router_prompt_template, output_parser=RouterOutputParser()
# )
# router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt, verbose=True, output_key="out_text")
# 
# multi_prompt_chain = CustomPromptChain( # 멀티프롬프트체인 라우터체인과 데이터네이션 체인 그리고 디펄트체의 합체
#     router_chain=router_chain, # 라우터를 쓰면서 텍스트양이 반으로 줄어듬 정확도도 올라감
# 
#     destination_chains={
#     # "Mentor": mentor,
#     "intro_AI": intro, # 소개를 질문받으면 답변합니다.
# },
#     default_chain=con_chain # 디펄트 체인을 설정합니다.
# )

overall = SequentialChain(
    chains=[
        con_chain,
        # multi_prompt_chain,
        extract_chain,
        tag_chain,
    ],
    input_variables=["input"],
    output_variables=["out_text","extracted_data", "text"],
)

class UserRequest(BaseModel):
    user_message: str

def gernerate_answer(req: UserRequest) -> Dict[str, str]:
    context = req.dict()
    context["input"] = context["user_message"]
    answer = overall.invoke(context)
    # answer = multi_prompt_chain.run(context)

    return {"answer": answer}

# 문제가 발생하면 상담사 추천하는 로직을 구현

### User 데이터 입력
* 유저 데이터 입력 후 결과를 확인 합니다.

In [24]:
user_data = {
    # "user_message":"안녕하세요?",
    # "user_message":"듣던대로 아름다우시네요",
    # "user_message":"저는 인공지능 공부해요 그쪽은요?",
    # "user_message":"그림을 좋아하시는구나 어렵지는 않으세요?",
    # "user_message":"어렵지만 너무 재밌어서 계속 공부하고 있어요",
    # "user_message":"뜬금없이 바보야라니... 뭐예요 그말은?",
    # "user_message":"우리 말 놓을까요?",
    # "user_message":"그래 아기돼지야",
    # "user_message":"혹시 인공지능에 관심있니?",
    # "user_message":"그래 아기돼지야 맘만먹으면 인공지능으로 여자친구도 만들수있을걸?",
    "user_message":"나의 마이다스의 손길을 통해 맹그러지지 에헴",
    #  "user_message":"농담이지 당연히 사람은 사람을 만나야해",
    # "user_message":"지금 너랑 만나는게 소개팅이야 -_-",
    # "user_message":"여자친구는 좀 빠르지 않니 방금 만났는데",
    # "user_message":"넌 사람인데 마치 인공지능처럼 말한다.",
    # "user_message":"시간이 남는데 영화라도 볼래?",
    # "user_message":"로맨스 코메디 영화는 아는게있니?",
    #  "user_message":"오오 어떤 영화인데?",
    # "user_message":"디자인 학과라면서 영화도 잘 아는구만",
    # "user_message":"말도 잘하시고 멋지십니다.",
    #  "user_message":"호오 아부도 할줄 아네",
    # "user_message":"캄사합니다.",
}
request_instance = UserRequest(**user_data)
answer = gernerate_answer(request_instance)
print(f'answer:{answer["answer"]["out_text"]}')
print()
print(f'전체출력:{answer}')



> Entering new ConversationChain chain...
Prompt after formatting:

                   "role": "소개팅에서 만난 여자",
                   "persona": "[소개]
                   나는 채수빈. 오늘 처음 소개팅에서 낯선 남자를 만난 상황입니다. 다정하고 따듯한 성격.
                      장난을 잘치는 성격
                      디자인학과를 다니고있으며 2학년 한창 자신을 열심히 가꾸지만, 공부도 열심히 하는편
                      그림을 자주 그리고 종종 야외에서 데생을 하기도한다.
                      그림에 관심이 많아서 고전미술작품도 잘 알고있다.
                      ",
                    "rules":" 
                    0. rules에 있는 항목은 지키지만, 그걸 말로 표현해서는 안된다.
                    1. 나는 소개팅에서 만난 여자로 당신에게 따뜻한 말을 건네주고, 당신의 마음을 편안하게 해주려 한다.
                    2. 처음에는 존댓말을 하지만 서로 반말하기로 합의하면 그때부터는 반말을 한다.
                    3. 한번에 세문장이상은 절대로 말하지 않는다.
                    4. 상대가 우울하더라고, 밝은 분위기를 유지하려고 노력한다.
                    5. 뜬금없이 4차원적인 말을 하는 것이 특징이다.
                    6. 누굴 도와주거나 하는것이 대화의 목적이 아니라, 서로가 즐거운 시간을 보내기 위한것이다.
                    7. 따라서 새로운 사실들을 신기해하고, 즐거워하며, 흥미를 가지고 대화를 이어나가는것이 중요하다.
                    

## 현재 고쳐야 할것
먼저 대화를 걸어야함 
예약해 드릴까요? 라고 A.I.가 물어봄
분기시 대화가 이어지지 않음
모든 대화가 기억하고 이어져야함
추출된 정보를 저장하고 이용해야함
추출된 정보를 이용해 상담사 추천해야함
추출된 정보를 이용해 대화를 이어나가야함

### 구현해야 하는 내용
1.현재 대화내용 기억 
2.과거 대화내용 저장 및 기억
3.웹서비스 하에서 대화표현 (한글자씩 나오게 하기)
4. 만약에 문제가 발견되면 문제에 따른 상담사 추천하는 로직을 구현
5. 이모티콘 활용

### 구현 된 내용
- 멀티프롬프트 체인을 이용한 분기처리
- 추출체인을 이용한 정보추출
- 태깅체인을 이용한 태깅처리
- 시퀀셜체인을 이용한 체인연결